<a href="https://colab.research.google.com/github/Martishamao/Sales_predictions/blob/main/Sales_Prediction_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 1 - Final (Core)
- Martisha Owens
- 9/21/2022



In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

filename = '/content/drive/MyDrive/Coding Dojo/05 Intro to Machine Learning/sales_predictions.csv'
df = pd.read_csv(filename)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [30]:
dfc = df.copy()

##Drop duplicates and fix inconsistencies in categorical data.

In [31]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [32]:
# Are there duplicates?
dfc.duplicated().sum()

0

> This shows that there are no duplicated data.


In [33]:
# Identify missing values.
dfc.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

> Item_Weight has 1463 missing data while Outlet_Size has 2410 missing data.

In [34]:
del dfc['Item_Identifier']
del dfc['Outlet_Size']

> Will delete Outlet_Size, this column has too many missing data. Will also delete Item_Identifier because this information does not  tell us anything valueable.

In [35]:
# Find and fix any inconsistent categories of data (example: fix cat, Cat, and cats so that they are consistent) 
dtypes = dfc.dtypes
dfc_obj = dtypes[dtypes=='object'].index
for i in dfc_obj:
  print(i)
  print(dfc[i].value_counts(dropna=False))
  print('\n\n')

Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64



Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64



Outlet_Identifier
OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    528
Name: Outlet_Identifier, dtype: int64



Outlet_Location_Type
Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64



Outlet_Type
Supermarket Type1  

Listed are the category and items that needs to be fixed.
- Item_Fat_Content = ['Low Fat' 'Regular' 'low fat' 'LF' 'reg']
- *Outlet_Size = ['Medium' 'Large' 'High' 'Small'] (Removed)*

In [36]:
dfc['Item_Fat_Content'].replace(['low fat', 'LF', 'reg'], ['Low Fat', 'Low Fat','Regular'], inplace = True)
print(dfc['Item_Fat_Content'].unique())

['Low Fat' 'Regular']


In [37]:
# For any numerical columns, obtain the summary statistics of each (min, max, mean)
dfc.describe(include=[np.number])

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


##Perform a Validation Split.

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn import set_config
set_config(display='diagram')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

In [39]:
X = dfc.drop(columns = 'Item_Outlet_Sales')
y = dfc['Item_Outlet_Sales']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [74]:
# Validation Check
X_perc = len(X_train)/(len(X_train)+len(X_test))*100
y_perc = len(y_train)/(len(y_train)+len(y_test))*100
print(f'Len of X train:{len(X_train)}, X test:{len(X_test)}, y train:{len(y_train)}, y test:{len(y_test)}.')
print(f'Split Ratio X:{round(X_perc,1)}% and y:{round(y_perc,1)}%')

Len of X train:6392, X test:2131, y train:6392, y test:2131.
Split Ratio X:75.0% and y:75.0%


## Identify each feature as numerical, ordinal, or nominal.
- *Item_Identifier, Nominal (Removed)*
- Item_Weight, Numerical
- Item_Fat_Content, Ordinal
- Item_Visibility, Numerical
- Item_Type, Nominal
- Item_MRP, Numerical
- Outlet_Identifier, Nominal
- Outlet_Establishment_Year, Numerical
- *Outlet_Size, Ordinal (Removed)*
- Outlet_Location_Type, Nominal 
- Outlet_Type, Nominal

In [75]:
Fat = {'Low Fat': 0, 'Regular': 1}

X_train['Item_Fat_Content'] = X_train['Item_Fat_Content'].replace(Fat)
X_test['Item_Fat_Content'] = X_train['Item_Fat_Content'].replace(Fat)

In [42]:
X_train.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type
4776,16.350,0,0.029565,Household,256.4646,OUT018,2009,Tier 3,Supermarket Type2
7510,15.250,1,0.000000,Snack Foods,179.7660,OUT018,2009,Tier 3,Supermarket Type2
5828,12.350,1,0.158716,Meat,157.2946,OUT049,1999,Tier 1,Supermarket Type1
5327,7.975,0,0.014628,Baking Goods,82.3250,OUT035,2004,Tier 2,Supermarket Type1
4810,19.350,0,0.016645,Frozen Foods,120.9098,OUT045,2002,Tier 2,Supermarket Type1


## Prepare preprocessing object

In [44]:
obj_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
scaler = StandardScaler()

In [45]:
median_imputer = SimpleImputer(strategy='median')
freq_imputer = SimpleImputer(strategy='most_frequent')

> Will use the median Simple Imputer to fill in the missing data for Item weight.

## Create a Model Pipeline.

In [46]:
obj_pipe = make_pipeline(freq_imputer, ohe)
obj_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [47]:
num_pipe = make_pipeline(median_imputer, scaler)
num_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [48]:
obj_tuple = (obj_pipe,obj_selector)
num_tuple = (num_pipe,num_selector)

preprocessor = make_column_transformer(obj_tuple,num_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f479e34c9d0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f479e3501d0>)])

# 1. Build a linear regression model to predict sales

### Build a linear regression model.

In [49]:
reg = LinearRegression()

reg_pipe = make_pipeline(preprocessor, reg)
reg_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f479e34c9d0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f479e3501d0>)])),
                ('linearregression', LinearRegression())])

In [50]:
reg_pipe.fit(X_train,y_train)

test_pred = reg_pipe.predict(X_test)
train_pred = reg_pipe.predict(X_train)

In [51]:
lineartest_df = pd.DataFrame(test_pred)
lineartrain_df = pd.DataFrame(train_pred)

###Evaluate the performance of your model based on r^2.

In [52]:
test_r2 = r2_score(y_test, test_pred)
train_r2 = r2_score(y_train, train_pred)
print(f'Train: {train_r2.round(3)} Test: {test_r2.round(3)}')

Train: 0.561 Test: 0.567


>The low Train and Test R2 indicates underfitting.

###Evaluate the performance of your model based on rmse.

In [53]:
train_MSE = mean_squared_error(y_train, train_pred)
test_MSE = mean_squared_error(y_test, test_pred)
train_RMSE = np.sqrt(train_MSE)
test_RMSE = np.sqrt(test_MSE)

print(f'train: {train_RMSE.round(4)} test: {test_RMSE.round(4)}')

train: 1139.3214 test: 1093.3209


#2. Your second task is to build a regression tree model to predict sales.

###Build a simple regression tree model.

In [100]:
dec_tree = DecisionTreeRegressor(random_state = 42)

dt_pipe = make_pipeline(preprocessor, dec_tree)
dt_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f479e34c9d0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f479e3501d0>)])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [101]:
dt_pipe.fit(X_train,y_train)

dt_test_pred = dt_pipe.predict(X_test)
dt_train_pred = dt_pipe.predict(X_train)

###Compare the performance of your model based on r^2.

In [102]:
dt_test_r2 = r2_score(y_test, dt_test_pred)
dt_train_r2 = r2_score(y_train, dt_train_pred)
print(f'Train: {dt_train_r2.round(3)} Test: {dt_test_r2.round(3)}')

Train: 1.0 Test: 0.219


>The model is severely overfitted, the model will need to be tuned in order to have a closer R2.

In [103]:
dec_tree.get_depth()

40

In [108]:
depths = list(range(2, 41)) 
scores = pd.DataFrame(index=depths, columns=['Test Score','Train Score'])
for depth in depths:
    dec_tree = DecisionTreeRegressor(max_depth=depth, random_state=42)
    dt_pipe = make_pipeline(preprocessor, dec_tree)
    dt_pipe.fit(X_train,y_train)
    train_score = dt_pipe.score(X_train, y_train)
    test_score = dt_pipe.score(X_test, y_test)
    scores.loc[depth, 'Train Score'] = train_score
    scores.loc[depth, 'Test Score'] = test_score

In [109]:
scores.sort_values(by='Test Score', ascending=False).head()

,Test Score,Train Score
5,0.594786,0.603933
4,0.584005,0.582625
6,0.583656,0.615153
7,0.577266,0.626837
8,0.559489,0.643864


In [110]:
dec_tree = DecisionTreeRegressor(max_depth = 5,random_state = 42)

dt_pipe = make_pipeline(preprocessor, dec_tree)

dt_pipe.fit(X_train,y_train)

dt_test_pred = dt_pipe.predict(X_test)
dt_train_pred = dt_pipe.predict(X_train)

In [111]:
dt_test_r2 = r2_score(y_test, dt_test_pred)
dt_train_r2 = r2_score(y_train, dt_train_pred)
print(f'Train: {dt_train_r2.round(3)} Test: {dt_test_r2.round(3)}')

Train: 0.604 Test: 0.595


>The Linear Regression R2 results were Train: 0.561 Test: 0.567. By the slightest margin, the tuned decision tree yielded better predictions.

###Compare the performance of your model based on rmse.


In [112]:
dt_train_MSE = mean_squared_error(y_train, dt_train_pred)
dt_test_MSE = mean_squared_error(y_test, dt_test_pred)
dt_train_RMSE = np.sqrt(dt_train_MSE)
dt_test_RMSE = np.sqrt(dt_test_MSE)

print(f'train: {dt_train_RMSE.round(4)} test: {dt_test_RMSE.round(4)}')

train: 1082.6551 test: 1057.3434


>The Linear Regression R2 results were Train: 1139.3214 Test: 1093.3209. Again, by the smallest margin, the tuned decision tree has a lower RMSE which indicates the outliers were not as large as linear.

#3. You need to determine which model to implement.
> Overall the R2 and RMSE for the Decision Tree was better therefore I would recommend the Tree Method. The Tree was however, tuned to yield these better results but that is part of the modeling process.